In [3]:
"""
Coffee News Scraper - Versão Final Simplificada
Coleta: Data, Hora, Título do resumo
"""

import requests
from bs4 import BeautifulSoup
import csv
import time
import re
from datetime import datetime


class CoffeeScraper:
    def __init__(self, min_year=2021, delay=2):
        """
        Args:
            min_year: Ano mínimo para coletar
            delay: Segundos entre páginas
        """
        self.min_year = min_year
        self.delay = delay
        self.session = requests.Session()
        self.session.headers.update({
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36'
        })
        self.current_page = 1
    
    def parse_date(self, date_str):
        """
        Converte data para DD/MM/YYYY e extrai ano
        
        Returns:
            tuple: (data_formatada, ano) ou (None, None)
        """
        if not date_str:
            return None, None
        
        date_str = date_str.strip()
        
        # Formato: DD/MM/YYYY ou DD/MM/YY
        match = re.search(r'(\d{1,2})/(\d{1,2})/(\d{2,4})', date_str)
        if match:
            day, month, year = match.groups()
            
            # Se ano com 2 dígitos, assumir 20XX
            if len(year) == 2:
                year = f"20{year}"
            
            return f"{day.zfill(2)}/{month.zfill(2)}/{year}", int(year)
        
        return None, None
    
    def is_date_valid(self, year):
        """Verifica se ano >= min_year"""
        if year is None:
            return True
        return year >= self.min_year
    
    def scrape_page(self, page_num):
        """
        Coleta notícias de uma página
        
        Returns:
            list: Lista de dicionários com notícias
        """
        base_url = "https://www.noticiasagricolas.com.br/noticias/cafe/"
        url = f"{base_url}?page={page_num}" if page_num > 1 else base_url
        
        print(f"\n[Página {page_num}] Acessando...")
        
        try:
            response = self.session.get(url, timeout=15)
            response.raise_for_status()
            soup = BeautifulSoup(response.content, 'html.parser')
            
            news_list = []
            
            # Encontrar blocos de data (ex: "09/01/2026")
            date_headers = soup.find_all('h3')
            
            for date_header in date_headers:
                # Extrair data do cabeçalho
                date_text = date_header.get_text(strip=True)
                data_formatada, year = self.parse_date(date_text)
                
                if not data_formatada:
                    continue
                
                # Verificar se data é válida (>= min_year)
                if not self.is_date_valid(year):
                    print(f"  ⏭️  Data {data_formatada} (ano {year}) antes de {self.min_year} - parando")
                    return news_list  # Retorna o que coletou até agora
                
                # Pegar lista de notícias desta data
                # A estrutura é: <h3>data</h3> seguido de <ul> com <li>s
                next_ul = date_header.find_next_sibling('ul')
                
                if not next_ul:
                    continue
                
                items = next_ul.find_all('li')
                
                print(f"  📅 {data_formatada}: {len(items)} notícias")
                
                for item in items:
                    try:
                        # Hora está no início do item (ex: "17:14")
                        text = item.get_text(strip=True)
                        
                        # Extrair hora (formato HH:MM)
                        time_match = re.search(r'(\d{1,2}):(\d{2})', text)
                        if not time_match:
                            continue
                        
                        hora = time_match.group(0)  # Ex: "17:14"
                        
                        # O título/resumo é o texto do <h2> dentro do <li>
                        h2_tag = item.find('h2')
                        if not h2_tag:
                            continue
                        
                        titulo = h2_tag.get_text(strip=True)
                        
                        # Adicionar à lista
                        news_list.append({
                            'data_hora': f"{data_formatada} {hora}",
                            'resumo': titulo
                        })
                        
                    except Exception as e:
                        print(f"    ⚠️  Erro ao processar item: {e}")
                        continue
            
            print(f"  ✅ Total coletado nesta página: {len(news_list)}")
            return news_list
            
        except Exception as e:
            print(f"  ❌ Erro ao acessar página: {e}")
            return []
    
    def save_checkpoint(self, page_num):
        """Salva página atual"""
        with open('checkpoint.txt', 'w') as f:
            f.write(str(page_num))
    
    def load_checkpoint(self):
        """Carrega última página"""
        try:
            with open('checkpoint.txt', 'r') as f:
                return int(f.read().strip())
        except:
            return 1
    
    def save_to_csv(self, news_list, filename='noticias_cafe.csv', mode='a'):
        """
        Salva notícias no CSV
        """
        if not news_list:
            return
        
        # Verificar se arquivo existe
        file_exists = False
        try:
            with open(filename, 'r'):
                file_exists = True
        except:
            pass
        
        with open(filename, mode, newline='', encoding='utf-8-sig') as f:
            writer = csv.DictWriter(f, fieldnames=['data_hora', 'resumo'])
            
            # Header apenas se novo ou sobrescrever
            if mode == 'w' or not file_exists:
                writer.writeheader()
            
            writer.writerows(news_list)
        
        print(f"  💾 Salvos {len(news_list)} registros em {filename}")
    
    def run(self, start_page=1, max_pages=None, csv_filename='noticias_cafe.csv'):
        """
        Executa o scraper
        """
        print("="*70)
        print("COFFEE NEWS SCRAPER - NOTÍCIAS AGRÍCOLAS")
        print("="*70)
        print(f"📅 Ano mínimo: {self.min_year}")
        print(f"📄 Página inicial: {start_page}")
        print(f"💾 Arquivo CSV: {csv_filename}")
        print("="*70)
        
        # Criar arquivo CSV (limpar)
        self.save_to_csv([], csv_filename, mode='w')
        
        page = start_page
        total_news = 0
        pages_without_news = 0
        stop_collecting = False
        
        while not stop_collecting:
            # Limite de páginas
            if max_pages and page > (start_page + max_pages - 1):
                print(f"\n✅ Limite de {max_pages} páginas atingido")
                break
            
            # Coletar página
            self.current_page = page
            news_list = self.scrape_page(page)
            
            # Salvar checkpoint
            self.save_checkpoint(page)
            
            if news_list:
                # Salvar no CSV
                self.save_to_csv(news_list, csv_filename, mode='a')
                total_news += len(news_list)
                pages_without_news = 0
            else:
                pages_without_news += 1
                print(f"  ⚠️  Página sem notícias válidas")
            
            # Parar se encontrou data anterior ao min_year
            if news_list:
                # Verificar se tem alguma data antiga demais
                for news in news_list:
                    date_str = news['data_hora'].split()[0]  # Pega só DD/MM/YYYY
                    _, year = self.parse_date(date_str)
                    
                    if year and year < self.min_year:
                        print(f"\n⏹️  Encontrou notícias de {year} < {self.min_year}. Parando.")
                        stop_collecting = True
                        break
            
            # Parar se 3 páginas vazias
            if pages_without_news >= 3:
                print(f"\n⏹️  3 páginas sem notícias. Fim da coleta.")
                break
            
            page += 1
            time.sleep(self.delay)
        
        print("\n" + "="*70)
        print("✅ COLETA CONCLUÍDA")
        print("="*70)
        print(f"📊 Total de notícias: {total_news}")
        print(f"📄 Páginas processadas: {page - start_page}")
        print(f"💾 Arquivo: {csv_filename}")
        print(f"📍 Checkpoint: página {self.current_page}")
        print("="*70)


def main():
    # ========== CONFIGURAÇÕES ==========
    MIN_YEAR = 2021          # Ano mínimo
    START_PAGE = 1           # Página inicial
    MAX_PAGES = None         # None = todas
    DELAY = 2                # Segundos entre páginas
    CSV_FILE = 'noticias_cafe.csv'
    # ===================================
    
    print("\n🚀 Coffee News Scraper")
    print(f"📅 Coletando desde {MIN_YEAR}\n")
    
    # Verificar checkpoint
    try:
        with open('checkpoint.txt', 'r') as f:
            last_page = int(f.read().strip())
            choice = input(f"💾 Checkpoint na página {last_page}. Continuar? (s/n): ")
            if choice.lower() == 's':
                START_PAGE = last_page + 1
                print(f"▶️  Continuando da página {START_PAGE}\n")
    except:
        pass
    
    # Executar
    scraper = CoffeeScraper(min_year=MIN_YEAR, delay=DELAY)
    
    try:
        scraper.run(
            start_page=START_PAGE,
            max_pages=MAX_PAGES,
            csv_filename=CSV_FILE
        )
    except KeyboardInterrupt:
        print("\n\n⚠️  Interrompido")
        print(f"💾 Checkpoint: página {scraper.current_page}")
        print(f"📄 Dados salvos em: {CSV_FILE}")


if __name__ == "__main__":
    main()


🚀 Coffee News Scraper
📅 Coletando desde 2021

COFFEE NEWS SCRAPER - NOTÍCIAS AGRÍCOLAS
📅 Ano mínimo: 2021
📄 Página inicial: 1
💾 Arquivo CSV: noticias_cafe.csv

[Página 1] Acessando...
  📅 09/01/2026: 3 notícias
  📅 08/01/2026: 4 notícias
  📅 07/01/2026: 3 notícias
  📅 06/01/2026: 2 notícias
  📅 05/01/2026: 2 notícias
  📅 02/01/2026: 2 notícias
  📅 30/12/2025: 4 notícias
  📅 29/12/2025: 2 notícias
  📅 26/12/2025: 3 notícias
  ✅ Total coletado nesta página: 25
  💾 Salvos 25 registros em noticias_cafe.csv

[Página 2] Acessando...
  📅 26/12/2025: 1 notícias
  📅 23/12/2025: 2 notícias
  📅 22/12/2025: 4 notícias
  📅 19/12/2025: 3 notícias
  📅 18/12/2025: 3 notícias
  📅 17/12/2025: 3 notícias
  📅 16/12/2025: 2 notícias
  📅 15/12/2025: 4 notícias
  📅 12/12/2025: 3 notícias
  ✅ Total coletado nesta página: 25
  💾 Salvos 25 registros em noticias_cafe.csv

[Página 3] Acessando...
  📅 12/12/2025: 1 notícias
  📅 11/12/2025: 3 notícias
  📅 10/12/2025: 4 notícias
  📅 09/12/2025: 2 notícias
  📅 08/12